In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [ ]:
import unsloth
from unsloth import FastLanguageModel
import torch

In [ ]:
max_seq_length = 8192
# dtype = 'torch.float16'
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    # device_map="cuda:0",
    dtype=torch.float16,
    load_in_4bit=load_in_4bit,
)


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(example):
    return {
        "text": alpaca_prompt.format(
            "Provide a helpful response to the user's mental health concern.",
            example["question"],
            example["answer"]
        ) + EOS_TOKEN
    }

from datasets import load_dataset

# Load the dataset
dataset = load_dataset("json", data_files="data_1.json")

# Apply formatting function
dataset = dataset.map(formatting_prompts_func)




In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 80,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Provide a helpful response to the user's query on gst law.", # instruction
        "What are the consequences if an Input Service Distributor distributes credit in excess to a recipient, and how is the excess credit recovered from the recipient along with interest?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 164, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
! git clone https://github.com/ggerganov/llama.cpp.git
! cd llama.cpp
! make

In [ ]:
model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q4_k_m")

In [ ]:
model.save_pretrained("GST_BOT_1") # Local saving

In [ ]:
!ls -lh GST_BOT_1/


In [ ]:
!zip -r lora_model.zip GST_BOT_1/
from google.colab import files
files.download("GST_BOT_1")



In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(GST_BOT_1) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Provide a helpful response to the user's query on gst law.", # instruction
        "What are the consequences if an Input Service Distributor distributes credit in excess to a recipient, and how is the excess credit recovered from the recipient along with interest?",  # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
import shutil

# Zip the directory
shutil.make_archive("GST_BOT_1", 'zip', "GST_BOT_1")


In [ ]:
from google.colab import files
files.download("GST_BOT_1.zip")
